# mimic LOB - DEMO


# 0. Imports

In [1]:
# Imports
from   IPython.display import display, HTML
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

# Display options
pd.set_option('display.width', 1000)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('mode.chained_assignment', None)

In [2]:
import mimicLOB as lob
from mimicLOB.orderbook.orderbook import OrderBook
from mimicLOB.agent.genericAgent import genericAgent

# 1. LOB creation

In [3]:
# b_tape = True means the LOB 
LOB = OrderBook(tick_size = 1, 
                b_tape    = True,
                verbose   = True)

# 2. Agents
The agents is constantly connected to the distant LOB via the server address.

#### Parameters :
1. distant : if true, the agent must have the server address. If False, he should have the orderbook object passed in the configuration.
2. server : when distant is True, the http address of the server the LOB is running on.
3. id : optional, default is generic. the if is used in the transaction tape.
4. b_record : optional, boolean, default is False. If True, the agent records all his activity : sent orders, executed trades, pending orders ...


In [4]:
EURONEXT_config = {'orderbook'  : LOB,
                   'id'         : 'market'}
                 
myself_config = {'orderbook'  : LOB,
                 'id'         : 'Fayçal',
                 'b_record'   : True}

otherguy_config = {'orderbook'  : LOB,
                   'id'         : 'Olivier'}

euronext = genericAgent(**EURONEXT_config)
myself   = genericAgent(**myself_config)
other    = genericAgent(**otherguy_config)

In [5]:
# Only agents with b_record = True
LOB.agentList

{'Fayçal': <mimicLOB.agent.genericAgent.genericAgent at 0x2b6d7b8b208>}

#### The agent with id 'market' can change the LOB configuration and reset it

In [6]:
euronext.setLOB_tickSize(1)

# 3. Get the LOB state
Either agents have access to LOB information

In [7]:
LOBstate = myself.getLOBState()
tickSize =  float(myself.getTickSize())
print(f'tick size : {tickSize}')
if type(LOBstate) == pd.DataFrame:
    if len(LOBstate) >0 :
        LOBstate = LOBstate.set_index('Price').sort_index()
        minPrice = LOBstate.index[0]
        maxPrice = LOBstate.index[-1]
        LOBstate = LOBstate.reindex(np.arange(minPrice, maxPrice+tickSize, tickSize))
        LOBstate.plot.bar(figsize=(20, 7))
    else:
        print('LOB is empty')
else:
    display(LOBstate)

tick size : 1.0
LOB is empty


# 4. Send orders

#### limit sell order @101, 102 & limit buy order @99, 100
Check the LOB state afterwards

In [8]:
# Send Orders
myself.send_sell_limit_order(quantity = 100,
                             price    = 101)

myself.send_sell_limit_order(quantity = 200,
                             price    = 102)

other.send_buy_limit_order(quantity   = 100,
                             price    = 100)

other.send_buy_limit_order(quantity   = 100,
                             price    = 99)
print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

**** I received this limit order **** 
: {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': Decimal('101'), 'trader_id': 'Fayçal', 'timestamp': 1586640951398290300}

**** I received this limit order **** 
: {'type': 'limit', 'side': 'ask', 'quantity': 200, 'price': Decimal('102'), 'trader_id': 'Fayçal', 'timestamp': 1586640951399288500}

**** I received this limit order **** 
: {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': Decimal('100'), 'trader_id': 'Olivier', 'timestamp': 1586640951399288500}

**** I received this limit order **** 
: {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': Decimal('99'), 'trader_id': 'Olivier', 'timestamp': 1586640951399288500}

Sent orders : 


,type,side,quantity,price,trader_id,timestamp,order_id
0,limit,ask,100,101,Fayçal,1586640951398290300,1
1,limit,ask,200,102,Fayçal,1586640951399288500,2




Pending orders (Orders that are still in the LOB) : 


,type,side,quantity,price,trader_id,timestamp,order_id
1,limit,ask,100,101,Fayçal,1586640951398290300,1
2,limit,ask,200,102,Fayçal,1586640951399288500,2




Executed Trades : 


""


#### market orders

We send a 150 sell market order. 100 will be executed @100 & 50 @99 

In [9]:
myself.send_sell_market_order(quantity   = 150)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

**** I received this market order **** 
: {'type': 'market', 'side': 'ask', 'quantity': 150, 'trader_id': 'Fayçal', 'timestamp': 1586640951422215800}

TRADE: Time - 1586640951422215800, Price - 100, Quantity - 100, TradeID - Olivier, Matching TradeID - Fayçal
TRADE: Time - 1586640951423212800, Price - 99, Quantity - 50, TradeID - Olivier, Matching TradeID - Fayçal
Sent orders : 


,type,side,quantity,price,trader_id,timestamp,order_id
0,limit,ask,100,101,Fayçal,1586640951398290300,1
1,limit,ask,200,102,Fayçal,1586640951399288500,2
2,market,ask,150,NaN,Fayçal,1586640951422215800,NaN




Pending orders (Orders that are still in the LOB) : 


,type,side,quantity,price,trader_id,timestamp,order_id
1,limit,ask,100,101,Fayçal,1586640951398290300,1
2,limit,ask,200,102,Fayçal,1586640951399288500,2




Executed Trades : 


,traded_price,traded_quantity,time,party1_id,party1_side,party1_order_id,party2_id,party2_side,party2_order_id
0,100,100,1586640951422215800,Olivier,bid,3,Fayçal,ask,None
1,99,50,1586640951423212800,Olivier,bid,4,Fayçal,ask,None


#### match orders
Olivier will buy 150 at 101. A transaction will happen, and the new best bid will be 101. Check LOB State.

The agent 'Fayçal' is notified of the transaction. Pending orders & transaction list are updated.

In [10]:
other.send_buy_limit_order(quantity = 150,
                           price    = 101)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

**** I received this limit order **** 
: {'type': 'limit', 'side': 'bid', 'quantity': 150, 'price': Decimal('101'), 'trader_id': 'Olivier', 'timestamp': 1586640951451139300}

TRADE: Time - 1586640951451139300, Price - 101, Quantity - 100, TradeID - Fayçal, Matching TradeID - Olivier
Sent orders : 


,type,side,quantity,price,trader_id,timestamp,order_id
0,limit,ask,0,101,Fayçal,1586640951398290300,1
1,limit,ask,200,102,Fayçal,1586640951399288500,2
2,market,ask,150,NaN,Fayçal,1586640951422215800,NaN




Pending orders (Orders that are still in the LOB) : 


,order_id,price,quantity,side,timestamp,trader_id,type
2,2,102,200,ask,1586640951399288500,Fayçal,limit




Executed Trades : 


,traded_price,traded_quantity,time,party1_id,party1_side,party1_order_id,party2_id,party2_side,party2_order_id
0,100,100,1586640951422215800,Olivier,bid,3,Fayçal,ask,None
1,99,50,1586640951423212800,Olivier,bid,4,Fayçal,ask,None
2,101,100,1586640951451139300,Fayçal,ask,1,Olivier,ask,None


# 5. Modify orders

Rules for modification : 
1. If the order is already executed : no new order
2. Any order entered into the Central Order Book may be modified or cancelled prior to its execution. Any increase in the order quantity or change in the limit price shall cause the forfeiture of time priority. (Rule 4202/4 of Euronext)

Rules for cancellation :
1. If the order is already executed : no order cancellation

#### modify an order

we modify an order to make it aggressive (from 102 to best bid = 101).
It will result in a traed of 50 shares + 150 shares will sit in the LOB at the best ask.

In [11]:
print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)



Pending orders (Orders that are still in the LOB) : 


,order_id,price,quantity,side,timestamp,trader_id,type
2,2,102,200,ask,1586640951399288500,Fayçal,limit


In [12]:
myself.modifyOrder(order_id     = 2, 
                   side         = 'ask',
                   new_price    = 101,
                   new_quantity = 200)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

**** I received this modify order **** 
: 2

**** I received this limit order **** 
: {'type': 'limit', 'side': 'ask', 'quantity': 200, 'price': Decimal('101'), 'trader_id': 'Fayçal', 'order_id': 2, 'timestamp': 1586640951492028800}

TRADE: Time - 1586640951492028800, Price - 101, Quantity - 50, TradeID - Olivier, Matching TradeID - Fayçal
Sent orders : 


,type,side,quantity,price,trader_id,timestamp,order_id
0,limit,ask,0,101,Fayçal,1586640951398290300,1
1,limit,ask,200,102,Fayçal,1586640951399288500,2
2,market,ask,150,NaN,Fayçal,1586640951422215800,NaN




Pending orders (Orders that are still in the LOB) : 


,order_id,price,quantity,side,timestamp,trader_id,type
2,2,101,150,ask,1586640951492028800,Fayçal,limit




Executed Trades : 


,traded_price,traded_quantity,time,party1_id,party1_side,party1_order_id,party2_id,party2_side,party2_order_id
0,100,100,1586640951422215800,Olivier,bid,3,Fayçal,ask,None
1,99,50,1586640951423212800,Olivier,bid,4,Fayçal,ask,None
2,101,100,1586640951451139300,Fayçal,ask,1,Olivier,ask,None
3,101,50,1586640951492028800,Olivier,bid,5,Fayçal,ask,None


#### Cancel an order

In [13]:
print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)



Pending orders (Orders that are still in the LOB) : 


,order_id,price,quantity,side,timestamp,trader_id,type
2,2,101,150,ask,1586640951492028800,Fayçal,limit


In [14]:
myself.cancelOrder(side     = 'ask', 
                   order_id = 2)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

**** I received this cancel order **** 
: 2



Pending orders (Orders that are still in the LOB) : 


""


# 6. Modify The LOB
One agent type (market) can reset the LOB, dump the tape, and modify the LOB properties 
#### Change tick size

In [15]:
euronext.setLOB_tickSize(0.1)

#### Remove all pending orders

In [16]:
euronext.resetLOB_PendingOrders()

'DONE'

#### Remove orders and transactions

In [17]:
euronext.resetLOB()

'DONE'